In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import transform

tf.logging.set_verbosity(tf.logging.INFO)

# Load data
x_train = np.load('exam2_train_x.npy').astype(np.float32)
y_train = np.load('exam2_train_y.npy').astype(int)
# x_train = x_train/255.

x_test = np.load('exam2_test_x.npy').astype(np.float32)
y_test = np.load('exam2_test_y.npy').astype(int)
# x_test = x_test/255.

In [2]:
def gen_augmented_data(x, y):
    
    new_x_imgs = []
    new_y = []
    for x_train, y_train in zip(x, y):

        cw = transform.rotate(image=x_train, angle=15., mode='edge', resize=False, preserve_range=True)
        ccw = transform.rotate(image=x_train, angle=-15., mode='edge', resize=False, preserve_range=True)

        # new_x_imgs.append(np.flipud(x_train))
        # new_y.append(y_train)
        new_x_imgs.append(np.fliplr(x_train))
        new_y.append(y_train)
        new_x_imgs.append(cw)
        new_y.append(y_train)
        new_x_imgs.append(ccw)
        new_y.append(y_train)

    return new_x_imgs, new_y

In [11]:
def conv_model(features, mode, labels):
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])
    
    c1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[5, 5], padding='same',
                          activation=tf.nn.relu)
    p1 = tf.layers.max_pooling2d(inputs=c1, pool_size=[2, 2], strides=2)
    
    c2 = tf.layers.conv2d(inputs=p1, filters=64, kernel_size=[5, 5], padding='same',
                          activation=tf.nn.relu)
    p2 = tf.layers.max_pooling2d(inputs=c2, pool_size=[2, 2], strides=2)
    
    # c3 = tf.layers.conv2d(inputs=p2, filters=128, kernel_size=[3, 3], padding='same',
    #                       activation=tf.nn.relu)
    # p3 = tf.layers.max_pooling2d(inputs=c3, pool_size=[2, 2], strides=2)
    
    # c4 = tf.layers.conv2d(inputs=p3, filters=256, kernel_size=[3, 3], padding='valid',
    #                       activation=tf.nn.relu)
    # p4 = tf.layers.max_pooling2d(inputs=c4, pool_size=[2, 2], strides=2)
    
    # c4 = tf.layers.conv2d(inputs=p3, filters=64, kernel_size=[4, 4], padding='same',
    #                       activation=tf.nn.relu, strides=1)
    # p4 = tf.layers.max_pooling2d(inputs=c4, pool_size=[2, 2], strides=1)
    
    a5 = tf.reshape(p2, [-1, 16*16*64])
    z5 = tf.layers.dense(a5, units=1024, activation=tf.nn.relu)
    z5 = tf.layers.dropout(inputs=z5, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    z6 = tf.layers.dense(z5, units=512, activation=tf.nn.relu)
    z6 = tf.layers.dropout(inputs=z6, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    z7 = tf.layers.dense(z6, units=6, activation=tf.nn.sigmoid)

    onehot = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=6)

    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot, logits=z7)
    
    predictions = {'classes': tf.argmax(input=z7, axis=1),
                   'probabilities': tf.nn.softmax(z7, name="softmax_tensor")}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, 
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
newx, newy = gen_augmented_data(x_train, y_train)
x_train = np.append(x_train, newx, axis=0).astype(np.float32)
y_train = np.append(y_train, newy, axis=0).astype(np.int)

print(x_train.shape)

x_train = x_train/255.

(4080, 64, 64, 3)


In [14]:
tf.set_random_seed(5)

print(x_train.shape)

clf = tf.estimator.Estimator(model_fn=conv_model, model_dir='model')

log_hook = tf.train.LoggingTensorHook(tensors={"probabilities": "softmax_tensor"}, every_n_iter=100)

train_input = tf.estimator.inputs.numpy_input_fn(x={'x': x_train}, 
                                                 y=y_train, batch_size=100, 
                                                 num_epochs=None, shuffle=True)
clf.train(input_fn=train_input, steps=400, hooks=[log_hook])

(4080, 64, 64, 3)
INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023589586630>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Restoring parameters from model\model.ckpt-200


INFO:tensorflow:Saving checkpoints for 201 into model\model.ckpt.


...

INFO:tensorflow:global_step/sec: 1.2026


INFO:tensorflow:probabilities = [[0.35207015 0.12958065 0.12957068 0.12963714 0.12957068 0.12957068]
 [0.35216892 0.12956293 0.12956369 0.12956242 0.12956372 0.12957828]
 [0.3292507  0.12163033 0.12141008 0.12140924 0.12141164 0.18488812]
 [0.12955435 0.35216266 0.12962289 0.1295534  0.1295534  0.1295534 ]
 [0.12938382 0.12935016 0.12935019 0.12935106 0.13110594 0.35145876]
 [0.12907995 0.129081   0.13311374 0.35049644 0.12908831 0.12914057]
 [0.12956248 0.12956248 0.12956254 0.12956248 0.35218734 0.12956262]
 [0.10722356 0.10722356 0.2874711  0.10736109 0.28349727 0.10722356]
 [0.12956251 0.12956251 0.35218742 0.12956251 0.12956251 0.12956251]
 [0.12956215 0.35218632 0.12956515 0.12956212 0.12956212 0.12956212]
 [0.12955928 0.12955928 0.12955928 0.12955928 0.3521786  0.1295843 ]
 [0.12931709 0.13122274 0.3514754  0.12931739 0.1293504  0.12931708]
 [0.1295597  0.1295597  0.12955974 0.12958159 0.12955971 0.35217956]
 [0.34998658 0.12877116 0.13498048 0.12875393 0.12875393 0.12875393]
 [

INFO:tensorflow:loss = 1.0846535, step = 501 (81.010 sec)


INFO:tensorflow:Saving checkpoints for 600 into model\model.ckpt.


INFO:tensorflow:Loss for final step: 1.090927.


In [15]:
predict_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_test}, y=y_test, num_epochs=1, shuffle=False)
results = clf.evaluate(input_fn=predict_input)

print(results)

INFO:tensorflow:Starting evaluation at 2018-04-15-08:20:44


INFO:tensorflow:Restoring parameters from model\model.ckpt-600


INFO:tensorflow:Finished evaluation at 2018-04-15-08:20:45


INFO:tensorflow:Saving dict for global step 600: accuracy = 0.92777777, global_step = 600, loss = 1.1324644


{'accuracy': 0.92777777, 'loss': 1.1324644, 'global_step': 600}
